# 1.0. IMPORTS

In [ ]:
!ipython --version

In [ ]:
# Libraries
import pandas as pd
import numpy as np
import inflection
import math
import seaborn as sns

from matplotlib import pyplot as plt
from IPython.display import Image

## 1.1. Helper Functions

## 1.2. Loading Data

In [ ]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# Merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

# 2.0. PASSO 01 - DESCRIÇÃO DOS DADOS

In [ ]:
df1 = df_raw.copy()

## 2.1. Rename Columns

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 
            'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 
            'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

# Rename
df1.columns = cols_new

## 2.2. Data Dimensions

In [ ]:
print('Numeber of rows {}'.format(df1.shape[0]))
print('Numeber of cols {}'.format(df1.shape[1]))

## 2.3. Data Types

In [ ]:
# Change format date
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

## 2.4. Check NA

In [ ]:
df1.isna().sum()

## 1.5. Fillout NA

In [ ]:
# Max value
df1['competition_distance'].max()

In [ ]:
# competition_distance 
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 2000000.0 if math.isnan(x) else x)

# competition_open_since_month
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) 
                                                else x['competition_open_since_month'], axis=1)

# competition_open_since_year                            
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) 
                                                else x['competition_open_since_year'], axis=1)

# promo2_since_week
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) 
                                                else x['promo2_since_week'], axis=1)
# promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) 
                                                else x['promo2_since_year'], axis=1)
# promo_interval      
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec', }

# Does not return value, changes directly in column
df1['promo_interval'].fillna(0, inplace=True)

df1['month_map'] = df1['date'].dt.month.map(month_map)

df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 
                                                             else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [ ]:
df1.isna().sum()

## 1.6. Change Types

In [ ]:
# Verify Data Types
df1.dtypes

In [ ]:
# Change data types
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)

## 1.7. Descriptive Statistical

### 1.7.1. Numerical Attributes

In [ ]:
# Numeric Data.
numeric_attributes = df1.select_dtypes(include=['int64', 'float64'])

# Central Tendency - Numeric Data: mean and median.
central_tendency_mean = pd.DataFrame(numeric_attributes.apply(np.mean)).T
central_tendency_median = pd.DataFrame(numeric_attributes.apply(np.median)).T

# Dispersion: std, min, max, range, skew and kurtosis.
dispersion_std = pd.DataFrame(numeric_attributes.apply(np.std)).T
dispersion_min = pd.DataFrame(numeric_attributes.apply(min)).T
dispersion_max = pd.DataFrame(numeric_attributes.apply(max)).T
dispersion_range = pd.DataFrame(numeric_attributes.apply(lambda x: x.max() - x.min())).T
dispersion_skew = pd.DataFrame(numeric_attributes.apply(lambda x: x.skew())).T
dispersion_kurtosis = pd.DataFrame(numeric_attributes.apply(lambda x: x.kurtosis())).T

# Concatenate
metrics = pd.concat([dispersion_min, dispersion_max, dispersion_range, dispersion_std,
                    central_tendency_mean, central_tendency_median, dispersion_skew, dispersion_kurtosis]).T.reset_index()

# Rename Columns
metrics.columns = ['Attributes', 'Min', 'Max', 'Range', 'Std', 'Mean', 'Median', 'Skew', 'Kurtosis']

In [ ]:
metrics

In [ ]:
sns.distplot(df1['competition_distance'])

### 1.7.2. Categorical Attributes

In [ ]:
# Categorical Data
categorical_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

In [ ]:
# Check Number of Categorical Levels
categorical_attributes.apply(lambda x: x.unique().shape[0])

In [ ]:
# Filter with effective sales.
effective_sales = df1[(df1['state_holiday'] != 0) & (df1['sales'] > 0)]

# Graphic
plt.subplot(1, 3, 1)
sns.boxplot(x='state_holiday', y='sales', data=effective_sales)

plt.subplot(1, 3, 2)
sns.boxplot(x='store_type', y='sales', data=effective_sales)

plt.subplot(1, 3, 3)
sns.boxplot(x='assortment', y='sales', data=effective_sales)

# 2.0. PASSO 02 - FEATURE ENGINEERING

## 2.1. Hypothesis Mind Map

In [ ]:
Image('img/mindmaphipotesis.png')

## 2.2. Hypothesis Creation

### 2.2.1. Store Hypotheses

1. Stores with more employees should sell more.
2. Stores with greater inventory capacity should sell more.
3. Larger stores should sell more.
4. Stores with larger assortments should sell more.
5. Stores with closer competitors should sell less.
6. Stores with longer competitors should sell more.

### 2.2.2. Product Hypotheses

1. Stores that invest more in Marketing should sell more.
2. Stores with more product exposure should sell more.
3. Stores with lower priced products should sell more.
5. Stores with more aggressive promotions (bigger discounts) should sell more.
6. Stores with longer running promotions should sell more.
7. Stores with more promotion days should sell more.
8. Stores with more consecutive promotions should sell more.

### 2.2.3. Time Hypotheses

1. Stores open during the Christmas holiday should sell more.
2. Stores should sell more over the years.
3. Stores should sell more in the second half of the year.
4. Stores should sell more after the 10th of each month.
5. Stores should sell less on weekends.
6. Stores should sell less during school holidays.

## 2.3. Final List of Hypotheses

1. Stores with larger assortments should sell more.
2. Stores with closer competitors should sell less.
3. Stores with longer competitors should sell more.
4. Stores with longer active promotions should sell more.
5. Stores with more promotion days should sell more.
6. Stores with more consecutive promotions should sell more.
7. Stores open during the Christmas holiday should sell more.
8. Stores should sell more over the years.
9. Stores should sell more in the second half of the year.
10. Stores should sell more after the 10th of each month.
11. Stores should sell less on weekends.
12. Stores should sell less during school holidays.